# Model Context Protocol (MCP)

In diesem Notebook werden wir verschiedene Techniken zum Kontextmanagement in LLMs erkunden. Wir werden verschiedene Memory-Typen in LangChain implementieren und vergleichen.

## 1. Grundlagen des Kontextmanagements

Beim Arbeiten mit LLMs ist das Kontextmanagement entscheidend, da alle Modelle ein begrenztes Kontextfenster haben. Dieses Fenster begrenzt, wie viel Information das Modell bei einer Anfrage berücksichtigen kann.

Hier einige wichtige Konzepte:
- **Kontextfenster**: Die maximale Anzahl an Tokens, die ein Modell verarbeiten kann
- **Token**: Eine Grundeinheit der Textverarbeitung (etwa 4 Zeichen im Englischen)
- **Memory**: Mechanismus zur Speicherung relevanter Teile einer Konversation

In [ ]:
# Benötigte Bibliotheken importieren
import os
from dotenv import load_dotenv
import time

# LangChain Importe
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

# Memory-Typen
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationSummaryMemory
from langchain.memory import ConversationBufferWindowMemory
from langchain.memory import ConversationTokenBufferMemory
from langchain.memory import VectorStoreRetrieverMemory
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# Umgebungsvariablen laden
load_dotenv()

# LLM initialisieren
llm = ChatOpenAI(model="gpt-3.5-turbo")
embeddings = OpenAIEmbeddings()

## 2. Kontextfenster verschiedener Modelle

Verschiedene LLMs haben unterschiedlich große Kontextfenster:

| Modell            | Kontextfenster | Besonderheiten                    |
|-------------------|----------------|-----------------------------------|
| GPT-3.5 Turbo     | 16K Tokens     | Kostengünstig                     |
| GPT-4 Turbo       | 128K Tokens    | Teurer, aber umfangreicherer Kontext |
| Claude 3 Opus     | 200K Tokens    | Sehr großes Kontextfenster        |
| Gemini Pro        | 32K Tokens     | Gutes Preis-Leistungs-Verhältnis  |
| Mistral Large     | 32K Tokens     | Open-Source Alternative           |

## 3. Token-Zählung verstehen

Lassen Sie uns untersuchen, wie Texte in Tokens umgewandelt werden. Dies ist wichtig, um das Kontextfenster effizient zu nutzen.

In [ ]:
import tiktoken

def count_tokens(text, model="gpt-3.5-turbo"):
    """Zählt die Anzahl der Tokens in einem Text für ein bestimmtes Modell."""
    encoding = tiktoken.encoding_for_model(model)
    tokens = encoding.encode(text)
    return len(tokens)

def show_token_breakdown(text, model="gpt-3.5-turbo"):
    """Zeigt die einzelnen Tokens eines Textes an."""
    encoding = tiktoken.encoding_for_model(model)
    tokens = encoding.encode(text)
    decoded_tokens = [encoding.decode_single_token_bytes(token).decode("utf-8", errors="replace") for token in tokens]
    return tokens, decoded_tokens

# Beispieltexte
text1 = "KI ist eine tolle Sache."
text2 = "Künstliche Intelligenz revolutioniert viele Bereiche unseres Lebens."

# Token-Anzahl ausgeben
print(f"Text 1: '{text1}'")
print(f"Anzahl Tokens: {count_tokens(text1)}")
print(f"Text 2: '{text2}'")
print(f"Anzahl Tokens: {count_tokens(text2)}")

# Token-Breakdown für ersten Text
tokens, decoded = show_token_breakdown(text1)
print("\nToken-Breakdown für Text 1:")
for i, (token, decoded_token) in enumerate(zip(tokens, decoded)):
    print(f"Token {i+1}: {token} -> '{decoded_token}'")

## 4. Memory-Typen in LangChain

LangChain bietet verschiedene Memory-Typen, die für unterschiedliche Anwendungsfälle optimiert sind.

### 4.1 Buffer Memory

Der einfachste Typ - speichert alle Nachrichten in voller Länge.

In [ ]:
buffer_memory = ConversationBufferMemory()

# Konversation hinzufügen
buffer_memory.save_context({"input": "Mein Name ist Hans"}, {"output": "Hallo Hans!"})
buffer_memory.save_context({"input": "Ich komme aus Berlin"}, {"output": "Berlin ist eine schöne Stadt."})
buffer_memory.save_context({"input": "Was ist mein Name?"}, {"output": "Dein Name ist Hans."})

# Abrufen des gesamten Kontexts
context = buffer_memory.load_memory_variables({})
print(context["history"])

### 4.2 Summary Memory

Erstellt eine fortlaufende Zusammenfassung der Konversation.

In [ ]:
summary_memory = ConversationSummaryMemory(llm=llm)

# Konversation hinzufügen
summary_memory.save_context({"input": "Mein Name ist Hans"}, {"output": "Hallo Hans!"})
summary_memory.save_context({"input": "Ich komme aus Berlin"}, {"output": "Berlin ist eine schöne Stadt."})
summary_memory.save_context({"input": "Was ist mein Name?"}, {"output": "Dein Name ist Hans."})
summary_memory.save_context({"input": "Ich interessiere mich für künstliche Intelligenz."}, 
                          {"output": "Das ist ein spannendes Thema! KI umfasst viele Bereiche wie Machine Learning, neuronale Netze und natürliche Sprachverarbeitung."})

# Zusammenfassung abrufen
summary = summary_memory.load_memory_variables({})
print("Zusammenfassung der Konversation:")
print(summary["history"])

### 4.3 Window Memory

Behält nur die letzten n Nachrichten bei.

In [ ]:
window_memory = ConversationBufferWindowMemory(k=2)  # Nur die letzten 2 Nachrichten behalten

# Konversation hinzufügen
window_memory.save_context({"input": "Mein Name ist Hans"}, {"output": "Hallo Hans!"})
window_memory.save_context({"input": "Ich komme aus Berlin"}, {"output": "Berlin ist eine schöne Stadt."})
window_memory.save_context({"input": "Was ist mein Name?"}, {"output": "Dein Name ist Hans."})

# Nur die letzten 2 Nachrichten abrufen
window_context = window_memory.load_memory_variables({})
print("Window Memory (letzte 2 Nachrichten):")
print(window_context["history"])

### 4.4 Token Buffer Memory

Begrenzt die Anzahl der gespeicherten Tokens.

In [ ]:
token_memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=100)

# Längere Konversation hinzufügen
token_memory.save_context({"input": "Mein Name ist Hans"}, {"output": "Hallo Hans!"})
token_memory.save_context({"input": "Erzähl mir etwas über künstliche Intelligenz"}, 
                        {"output": "Künstliche Intelligenz (KI) ist ein Teilgebiet der Informatik, das sich mit der Automatisierung intelligenten Verhaltens befasst. KI-Systeme können lernen, Probleme lösen und Entscheidungen treffen."})
token_memory.save_context({"input": "Was sind Anwendungsbeispiele?"}, 
                        {"output": "Anwendungsbeispiele für KI sind Spracherkennung, autonomes Fahren, medizinische Diagnose, Bilderkennung und natürlich Sprachmodelle wie ich."})

# Token-begrenzte Geschichte abrufen
token_context = token_memory.load_memory_variables({})
print("Token Buffer Memory (max. 100 Tokens):")
print(token_context["history"])

# Anzahl der Tokens in der Geschichte
print(f"\nAnzahl Tokens im Memory: {count_tokens(token_context['history'])}")

### 4.5 Vector Store Memory

Speichert die Konversation in einer Vektordatenbank und ruft die relevantesten Nachrichten basierend auf der aktuellen Anfrage ab.

In [ ]:
# Einfache Vektordatenbank erstellen
vectorstore = Chroma(embedding_function=embeddings, collection_name="memory_store")
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# Vector Memory initialisieren
vector_memory = VectorStoreRetrieverMemory(retriever=retriever)

# Verschiedene Informationen hinzufügen
vector_memory.save_context({"input": "Mein Name ist Hans"}, {"output": "Hallo Hans!"})
vector_memory.save_context({"input": "Ich komme aus Berlin"}, {"output": "Berlin ist eine schöne Stadt."})
vector_memory.save_context({"input": "Meine Lieblingsfarbe ist blau"}, {"output": "Blau ist eine schöne Farbe."})
vector_memory.save_context({"input": "Ich mag Pizza und Pasta"}, {"output": "Italienisches Essen ist sehr beliebt."})
vector_memory.save_context({"input": "Ich arbeite als Softwareentwickler"}, {"output": "Softwareentwicklung ist ein spannendes Berufsfeld."})

# Relevanten Kontext für eine Anfrage abrufen
query_info = vector_memory.load_memory_variables({"prompt": "Was war noch mal mein Name und woher komme ich?"})
print("Vector Memory (relevante Nachrichten basierend auf der Abfrage):")
print(query_info["history"])

## 5. Memory-Typen in der Praxis vergleichen

In [ ]:
def create_conversation_chain(memory, name):
    """Erstellt eine Konversationskette mit dem angegebenen Memory-Typ."""
    return ConversationChain(
        llm=llm, 
        memory=memory,
        verbose=True
    )

# Verschiedene Memory-Typen erstellen
buffer_memory = ConversationBufferMemory()
summary_memory = ConversationSummaryMemory(llm=llm)
window_memory = ConversationBufferWindowMemory(k=2)

# Konversationsketten erstellen
buffer_chain = create_conversation_chain(buffer_memory, "Buffer Memory")
summary_chain = create_conversation_chain(summary_memory, "Summary Memory")
window_chain = create_conversation_chain(window_memory, "Window Memory")

# Funktion zum Ausführen und Messen einer Konversation
def run_conversation(chain, name, messages):
    """Führt eine Konversation mit der angegebenen Kette durch und misst die Antwortzeit."""
    print(f"\n=== {name} ===\n")
    responses = []
    times = []
    
    for i, message in enumerate(messages):
        print(f"Nachricht {i+1}: {message}")
        start_time = time.time()
        response = chain.predict(input=message)
        end_time = time.time()
        elapsed = end_time - start_time
        
        responses.append(response)
        times.append(elapsed)
        
        print(f"Antwort: {response}")
        print(f"Zeit: {elapsed:.2f} Sekunden\n")
    
    return responses, times

In [ ]:
# Testreihe mit verschiedenen Fragen
test_messages = [
    "Mein Name ist Maria und ich komme aus München.",
    "Ich bin 35 Jahre alt und arbeite als Ärztin.",
    "In meiner Freizeit spiele ich gerne Klavier.",
    "Ich habe zwei Kinder, Thomas und Lisa.",
    "Kannst du mir sagen, wie ich heiße und was ich beruflich mache?"
]

# Konversation mit verschiedenen Memory-Typen durchführen
buffer_responses, buffer_times = run_conversation(buffer_chain, "Buffer Memory", test_messages)
summary_responses, summary_times = run_conversation(summary_chain, "Summary Memory", test_messages)
window_responses, window_times = run_conversation(window_chain, "Window Memory", test_messages)

## 6. Vergleich der Antwortqualität

In [ ]:
# Vergleich der letzten Antworten
print("Vergleich der letzten Antworten (Frage: Kannst du mir sagen, wie ich heiße und was ich beruflich mache?)")
print("\nBuffer Memory Antwort:")
print(buffer_responses[-1])
print("\nSummary Memory Antwort:")
print(summary_responses[-1])
print("\nWindow Memory Antwort:")
print(window_responses[-1])

# Vergleich der Antwortzeiten
print("\nDurchschnittliche Antwortzeiten:")
print(f"Buffer Memory: {sum(buffer_times)/len(buffer_times):.2f} Sekunden")
print(f"Summary Memory: {sum(summary_times)/len(summary_times):.2f} Sekunden")
print(f"Window Memory: {sum(window_times)/len(window_times):.2f} Sekunden")

## 7. Manuelle Implementation eines hierarchischen Gedächtnisses

Bei komplexen Anwendungen kann ein hierarchischer Ansatz sinnvoll sein, der verschiedene Memory-Typen kombiniert.

In [ ]:
class HierarchicalMemory:
    def __init__(self, llm):
        # Kurzzeit-Gedächtnis: die letzten n Nachrichten (Window Memory)
        self.short_term = ConversationBufferWindowMemory(k=3)
        
        # Mittelfristiges Gedächtnis: Zusammenfassung bisheriger Konversation
        self.medium_term = ConversationSummaryMemory(llm=llm)
        
        # Langzeit-Gedächtnis: Wichtige Fakten in strukturierter Form
        self.long_term_facts = {}
        
        self.llm = llm
    
    def save_context(self, input_text, output_text):
        # Speichern im Kurzzeit- und mittelfristigen Gedächtnis
        self.short_term.save_context({"input": input_text}, {"output": output_text})
        self.medium_term.save_context({"input": input_text}, {"output": output_text})
        
        # Wichtige Fakten extrahieren und im Langzeitgedächtnis speichern
        self.extract_facts(input_text)
    
    def extract_facts(self, input_text):
        # Einfache Faktenextraktion durch Schlüsselwörter
        if "mein Name ist" in input_text.lower():
            parts = input_text.split("mein Name ist", 1)
            if len(parts) > 1:
                name = parts[1].split(".")[0].strip()
                self.long_term_facts["name"] = name
        
        if "ich komme aus" in input_text.lower():
            parts = input_text.split("ich komme aus", 1)
            if len(parts) > 1:
                location = parts[1].split(".")[0].strip()
                self.long_term_facts["location"] = location
        
        if "ich arbeite als" in input_text.lower():
            parts = input_text.split("ich arbeite als", 1)
            if len(parts) > 1:
                job = parts[1].split(".")[0].strip()
                self.long_term_facts["job"] = job
    
    def get_relevant_context(self, query):
        # Kontext aus verschiedenen Quellen zusammenstellen
        context = ""
        
        # 1. Wichtige Fakten aus dem Langzeitgedächtnis
        if self.long_term_facts:
            context += "Wichtige Fakten:\n"
            for key, value in self.long_term_facts.items():
                context += f"- {key.capitalize()}: {value}\n"
        
        # 2. Zusammenfassung der bisherigen Konversation
        medium_context = self.medium_term.load_memory_variables({})
        if medium_context["history"]:
            context += "\nZusammenfassung der Konversation:\n"
            context += medium_context["history"]
        
        # 3. Letzte Nachrichten aus dem Kurzzeitgedächtnis
        short_context = self.short_term.load_memory_variables({})
        if short_context["history"]:
            context += "\nLetzte Nachrichten:\n"
            context += short_context["history"]
        
        return context

In [ ]:
# Hierarchisches Gedächtnis testen
hierarchical_memory = HierarchicalMemory(llm)

# Konversation simulieren
conversation = [
    ("Mein Name ist Klaus Schmidt.", "Hallo Klaus, schön dich kennenzulernen!"),
    ("Ich komme aus Hamburg.", "Hamburg ist eine schöne Stadt am Wasser."),
    ("Ich arbeite als Softwareentwickler.", "Softwareentwicklung ist ein spannender Beruf mit vielen Möglichkeiten."),
    ("Ich interessiere mich für künstliche Intelligenz.", "Das Gebiet der KI entwickelt sich rasant und bietet viele interessante Anwendungsmöglichkeiten."),
    ("Ich habe letztes Jahr ein Projekt mit Python umgesetzt.", "Python ist eine großartige Sprache für KI-Projekte und viele andere Anwendungen."),
]

for input_text, output_text in conversation:
    hierarchical_memory.save_context(input_text, output_text)
    print(f"User: {input_text}")
    print(f"AI: {output_text}\n")

# Relevanten Kontext für eine Anfrage abrufen
query = "Wie heißt du und was machst du beruflich?"
context = hierarchical_memory.get_relevant_context(query)
print("=== Hierarchisches Gedächtnis ===\n")
print(context)

# Langzeitgedächtnis überprüfen
print("\n=== Langzeitgedächtnis (extrahierte Fakten) ===\n")
for key, value in hierarchical_memory.long_term_facts.items():
    print(f"{key.capitalize()}: {value}")

## 8. Praktische Anwendung: Chat mit effizientem Kontextmanagement

In [ ]:
def create_optimized_chat():
    # Für einen effizienten Chat kombinieren wir verschiedene Memory-Typen
    
    # System-Prompt definieren
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Du bist ein hilfreicher Assistent. Verwende die folgenden Informationen, um Fragen zu beantworten:\n{context}"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}")
    ])
    
    # Hierarchisches Gedächtnis für effizienten Kontext
    memory = HierarchicalMemory(llm)
    
    # Chat-Funktion erstellen
    def chat(message):
        context = memory.get_relevant_context(message)
        
        # Konversationshistorie aus dem Kurzzeitgedächtnis laden
        history = memory.short_term.load_memory_variables({})["history"]
        
        # LLM-Anfrage stellen
        messages = [
            SystemMessage(content=f"Du bist ein hilfreicher Assistent. Verwende die folgenden Informationen, um Fragen zu beantworten:\n{context}"),
            HumanMessage(content=message)
        ]
        response = llm.invoke(messages).content
        
        # Kontext aktualisieren
        memory.save_context(message, response)
        
        return response
    
    return chat

# Chat-Funktion erstellen
chat = create_optimized_chat()

# Beispiel-Konversation durchführen
messages = [
    "Hallo! Ich bin Julia und komme aus München.",
    "Ich bin 32 Jahre alt und arbeite als Grafikdesignerin.",
    "Ich habe einen Hund namens Bello und eine Katze namens Mimi.",
    "In meiner Freizeit gehe ich gerne wandern und fotografieren.",
    "Kannst du mir zusammenfassen, was du über mich weißt?"
]

for message in messages:
    print(f"\nUser: {message}")
    response = chat(message)
    print(f"AI: {response}")

## 9. Best Practices für Kontextmanagement

1. **Relevanz priorisieren**: Nicht jede Information ist gleich wichtig. Priorisieren Sie kritische Informationen wie Nutzerdaten, Präferenzen und aktuelle Anfragen.

2. **Hierarchischen Ansatz verwenden**: Kombinieren Sie verschiedene Memory-Typen für verschiedene Informationsebenen:
   - **Kurzzeitgedächtnis**: Aktuelle Konversation (letzte 3-5 Nachrichten)
   - **Mittelfristiges Gedächtnis**: Zusammenfassung der bisherigen Konversation
   - **Langzeitgedächtnis**: Wichtige Fakten und Präferenzen

3. **Kontext regelmäßig komprimieren**: Verwenden Sie Summary Memory oder LLM-basierte Zusammenfassungen, um lange Konversationen zu komprimieren.

4. **Dynamische Kontexterweiterung**: Laden Sie nur dann externe Informationen, wenn sie für die aktuelle Anfrage relevant sind.

5. **Token-Überwachung**: Überwachen Sie die Anzahl der verwendeten Tokens, um Kosten zu kontrollieren und das Kontextfenster optimal zu nutzen.

6. **Metadaten nutzen**: Speichern Sie Zeitstempel und Wichtigkeitsbewertungen für jede Information, um später besser filtern zu können.

7. **Vergessen implementieren**: Entfernen Sie weniger relevante oder veraltete Informationen aus dem Kontext.

## 10. Fazit

Effektives Kontextmanagement ist entscheidend für die Leistung von LLM-Anwendungen. Die richtige Strategie hängt vom konkreten Anwendungsfall ab:

- **Buffer Memory**: Gut für kurze Konversationen mit vollständigem Kontext
- **Summary Memory**: Sinnvoll für längere Konversationen, bei denen Details weniger wichtig sind
- **Window Memory**: Effizient für aufgabenorientierte Dialoge, wo nur der aktuelle Kontext wichtig ist
- **Vector Memory**: Ideal für sehr lange Konversationen oder große Wissensdatenbanken
- **Hierarchischer Ansatz**: Die beste Lösung für komplexe Anwendungen mit unterschiedlichen Kontextanforderungen

Durch die richtige Kombination dieser Techniken können Sie die Effektivität Ihrer LLM-basierten Anwendungen maximieren und gleichzeitig Kosten und Latenz minimieren.